# Exploring the first release of the panelapp data


Made available on: 2020.10.01


**Conclusions:**

* Number of total assoc: 60k
* NUmber of significant assoc: 42K (panel version > 1.0 & green or amber)
* 1k of the sigificant assoc is missing phenotypes
* 48k total phenotypes (~1.2 phenotype per assoc)
* 27k unique terms.
* Out of 100 randomly selected phenotypes 96 could be mapped, 13 with exact match.

In [7]:
import json 
import pandas as pd
import gzip

panelapp_file = '/Users/dsuveges/project/evidences/All_genes_20200928-1959.tsv'

panelapp_df = pd.read_csv(panelapp_file, sep='\t')
print(f'number of associations: {len(panelapp_df)}')

# The data needs to be filterd for the following criteria:
# 1. panel version >= 1.0
# 2. association is green or amber

panelapp_signif = panelapp_df.loc[(panelapp_df['Panel Version'] >= 1) &
                                 ((panelapp_df['List'] == 'green') |
                                 (panelapp_df['List'] == 'amber'))]
print(f'number of significant associations: {len(panelapp_signif)}')
print(f'number of genes in the signif set: {len(panelapp_signif["EnsemblId(GRch38)"].unique())}')


number of associations: 60772
number of significant associations: 42371
number of genes in the signif set: 4002


### What about the phenotypes?

In [21]:
# Significant associations with no phenotypes:
print(f'Missing phenotypes annotation: {len(panelapp_signif.loc[panelapp_signif.Phenotypes.isna()])}')


Missing phenotypes annotation: 1072


In [29]:
# Containers for phenotypes;
phenotypes = []

# Filter out unique set of phenotypes:
phenotype_series = pd.Series(
    panelapp_df.
    loc[~panelapp_df.Phenotypes.isna()]
    .Phenotypes
    .unique()
)

# Add phenotypes:
phenotype_series.apply(lambda x: [phenotypes.append(y) for y in x.split(';') if isinstance(x, str)])

print(f'Total number of phenotype terms: {len(phenotypes)}')
print(f'number of unique phenotype terms: {len(set(phenotypes))}')

Total number of phenotype terms: 48446
number of unique phenotype terms: 27173


In [33]:
# from ontoma import OnToma
import random


otmap = OnToma()

INFO     - ontoma.downloaders - ZOOMA to EFO mappings - Parsed 3666 rows
INFO     - ontoma.downloaders - OMIM to EFO mappings - Parsed 8215 rows


In [54]:
mappings = []
for phenotype in random.sample(phenotypes, 100):
    mappings.append(otmap.find_term(phenotype,verbose=True))

mappings_df = pd.DataFrame([x for x in mappings if x is not None ])
print(f'mapping found for {len(mappings_df)} terms.')
print(f'exact match found for {len(mappings_df.loc[mappings_df.quality == "match"])}')

WARNING  - ontoma.interface - Found http://www.ebi.ac.uk/efo/EFO_0002508 for 616710 from OLS API EFO lookup - fuzzy - check if valid
INFO     - ontoma.interface - Found http://www.orpha.net/ORDO/Orphanet_183660 for SCID from OLS API EFO lookup - match - None
WARNING  - ontoma.interface - Found http://www.orpha.net/ORDO/Orphanet_811 for Shwachman-Diamond syndrome, 260400 from OLS API EFO lookup - fuzzy - check if valid
WARNING  - ontoma.interface - Found http://www.ebi.ac.uk/efo/EFO_1000952 for Glycogen storage disease type VI, Hers (Glycogen storage disorders) from OLS API EFO lookup - fuzzy - check if valid
WARNING  - ontoma.interface - Found http://www.orpha.net/ORDO/Orphanet_417 for Transient Neonatal Hyperparathyroidism from OLS API EFO lookup - fuzzy - check if valid
WARNING  - ontoma.interface - Found http://www.orpha.net/ORDO/Orphanet_397937 for Polyglucosan body myopathy 1 with or without immunodeficiency MIM#615895 from OLS API EFO lookup - fuzzy - check if valid
WARNING  - on

WARNING  - ontoma.interface - Found http://purl.obolibrary.org/obo/MONDO_0007415 for Complex IV Mitochondrial respiratory chain disorders caused by nuclear variants only from OLS API EFO lookup - fuzzy - check if valid
INFO     - ontoma.interface - Found http://purl.obolibrary.org/obo/HP_0001263 for Global developmental delay from Zooma API lookup - match - None
ERROR    - ontoma.interface - Could not find *any* term for string: Erythremias, beta-
WARNING  - ontoma.interface - Found http://www.ebi.ac.uk/efo/EFO_0007821 for [Hair morphology 1, hair thickness], 612630 -3 from OLS API EFO lookup - fuzzy - check if valid
WARNING  - ontoma.interface - Found http://www.orpha.net/ORDO/Orphanet_1475 for Papillorenal syndrome, AR from OLS API EFO lookup - fuzzy - check if valid
WARNING  - ontoma.interface - Found http://www.orpha.net/ORDO/Orphanet_500 for LEOPARD syndrome 2 611554 from OLS API EFO lookup - fuzzy - check if valid
WARNING  - ontoma.interface - Found http://www.ebi.ac.uk/efo/EFO_0

WARNING  - ontoma.interface - Found http://www.orpha.net/ORDO/Orphanet_324588 for  Myokymia from OLS API EFO lookup - fuzzy - check if valid
WARNING  - ontoma.interface - Found http://www.ebi.ac.uk/efo/EFO_0002430 for Myelofibrosis, somatic, 254450 from OLS API EFO lookup - fuzzy - check if valid
WARNING  - ontoma.interface - Found http://www.orpha.net/ORDO/Orphanet_324588 for Familial dyskinesia 606703 from OLS API EFO lookup - fuzzy - check if valid
WARNING  - ontoma.interface - Found http://purl.obolibrary.org/obo/MONDO_0024464 for Septooptic dysplasia, 182230Pituitary hormone deficiency, combined, 5, 182230Growth hormone deficiency with pituitary anomalies, 182230 from OLS API EFO lookup - fuzzy - check if valid
WARNING  - ontoma.interface - Found http://www.orpha.net/ORDO/Orphanet_207 for Craniosynostosis, nonspecific Crouzon syndrome 123500 from OLS API EFO lookup - fuzzy - check if valid
WARNING  - ontoma.interface - Found http://purl.obolibrary.org/obo/MONDO_0001700 for Megalob

mapping found for 96 terms.
exact match found for 13


In [56]:
panelapp_df.columns

Index(['Symbol', 'Panel Id', 'Panel Name', 'Panel Version', 'Panel Status',
       'List', 'Sources', 'Mode of inheritance', 'Mode of pathogenicity',
       'Tags', 'EnsemblId(GRch37)', 'EnsemblId(GRch38)', 'HGNC', 'Biotype',
       'Phenotypes', 'GeneLocation((GRch37)', 'GeneLocation((GRch38)',
       'Panel Types', 'Super Panel Id', 'Super Panel Name',
       'Super Panel Version'],
      dtype='object')